In [ ]:
import pyc3dserver as c3d
itf = c3d.c3dserver(False)

import pandas as pd
import math
import numpy as np

def getData(file_name, number=0, close_other=True):#les données intéressantes du fichier c3d
    if close_other:#sait pas si ça marche
        ret=c3d.close_c3d(itf)
    ret = c3d.open_c3d(itf, file_name)

    n_frames = c3d.get_last_frame(itf)
    lab=c3d.get_marker_names(itf)
    data=c3d.get_dict_markers(itf)['DATA']['POS']
    
    ret=c3d.close_c3d(itf)
    
    #Si toutes les noms des trajectoires commencent avec les memes caractères (ex: New), on supprime ces caractères redondants
    for i in range(1, len(lab[0])):
        if all(list(t[0:i]==lab[0][0:i] for t in lab)):
            pass;
        else:
            i-=1
            lab=[e[i:] for e in lab]
            datak=[e[i:] for e in data.keys()]
            data=dict(zip(datak, list(data.values())))
            break;
        
    
    labdico = {}
    for i in lab:
        df = pd.DataFrame(data[i], columns=['x', 'y', 'z']).apply(pd.to_numeric)
        df = df[(df.x!=0) & (df.y!=0) & (df.z!=0)]
        if not df.empty:
            labdico[i]=df.copy()       

    return n_frames, list(labdico.keys()), labdico  
    

#vrai si le marqueur lab commence à la frame
def is_new(lab, frame):
    return labdico[lab].index[0]==frame

#vrai si le marqueur lab fini à la frame
def is_old(lab, frame):
    return labdico[lab].index[-1]==frame

#vrai si le marqueur lab existe à la frame
def is_present(lab, frame):
    return frame in lab.index

#liste les marqueurs présents à la frame
def markers_present(frame):
    mark=[]
    for i in labdico:
        if is_present(labdico[i], frame):
            mark.append(i)
    return mark
    
#retourne le nombre de marqueur présents à la frame
def n_markers_present(frame):
    n=0
    for i in labdico:
        if is_present(labdico[i], frame):
            n+=1
    return n
    
#vrai si 2 marqueurs (représentés par des dataframes) se superposent
def superpos(df1, df2):
    if (df1.index[-1]<df2.index[0]) or (df2.index[-1]<df1.index[0]):
        return False
    else: return True
    

def avg_speed(traj, step):
    if len(labdico[traj])>step:
        cpt=0
        speed=0
        for frame in range(labdico[traj].index[0], labdico[traj].index[-1]-step, step):
            pos_future=labdico[traj].loc[frame+step]
            speed+=math.dist(labdico[traj].loc[frame], pos_future)
            cpt+=1
        return speed/(cpt*step)
    else:
        return 'step error'

def get_key_from_value(val):
    keys = [k for k, v in connect.items() if val in v]
    if keys:
        return keys[0]
    return None

def avg_speed(traj, step):
    if len(labdico[traj])>step:
        cpt=0
        speed=0
        for frame in range(labdico[traj].index[0], labdico[traj].index[-1]-step, step):
            pos_future=labdico[traj].loc[frame+step]
            speed+=math.dist(labdico[traj].loc[frame], pos_future)
            cpt+=1
        return speed/(cpt*step)
    else:
        return 'step error'

In [ ]:
#LIster ici les artefacts
artefacts = { #size=109
  'Measurement03': ['0041', '0070', '0079'],
  'Measurement04': ['0015', '0037', '0062', '0066', '0091', '0097', '0104', '0105', '0106', '0109', '0110', '0113', '0115', '0116', '0122', '0125', '0129', '0130', '0131', '0134', '0136', '0147', '0157', '0159'],
  'Measurement05': ['0014', '0040', '0045', '0051', '0054', '0064', '0089', '0090', '0091', '0097', '0102'],
  'Measurement06': ['0098'],
  'Measurement07': ['0074', '0104', '0105', '0123', '0124', '0125', '0126', '0136', '0141', '0142', '0143', '0153', '0154', '0155', '0156', '0169', '0171'],
  'Measurement08': ['0041', '0042', '0062', '0071', '0078', '0079', '0080', '0084', '0085', '0086'],
  'Measurement09': ['0103', '0117', '0118'], 
  'Measurement10': ['0083', '0092', '0097', '0098', '0104', '0105', '0109', '0110'],
  'Measurement11': ['0011', '0060', '0068', '0074', '0075', '0079', '0123', '0134', '0146', '0147', '0165', '0177', '0180'],
  'Measurement12': ['0013', '0085', '0086', '0094', '0095', '0096', '0102', '0103', '0104', '0112', '0113', '0114', '0117', '0119', '0120', '0129', '0130', '0131', '0144']
}

In [ ]:
cd C:\Users\spear\Documents\Qualisys\Data

In [ ]:
def find_artifacts1(labdico, step, dist_max):#Supprime les marqueurs loin des autres en regardant toute les "artif_detec_step1" frames
    
    artef=[]
    frames_seen=[]
    
    for frame in range(step ,n_frames-step, step):
        mark_pres=markers_present(frame)
        #On dégage les marqueurs distants:
        for i in mark_pres:
            distant=True
            for j in mark_pres:
                if i!=j:
                    if math.dist(labdico[i].loc[frame], labdico[j].loc[frame]) < dist_max:
                        distant=False
                        break
            if distant:
                artef.append(i)
                
    return sorted(list(dict.fromkeys(artef)))

#===============================================================================================
def find_artifacts2(labdico, dist_max):#Supprime les marqueurs si loins des autres a leur naissance et mort
    
    artef=[]
    frames_seen=[]
    
    for r in [0, -1]:#0 pour la première frame, -1 pour la dernière
        frames_seen=[]
        for h in labdico:
            if not labdico[h].index[r] in frames_seen: #si n'a pas été vu:
                frames_seen.append(labdico[h].index[r]) #on l'ajoute à la liste, frame_seen[-1] devient la frame intéressante
                mark_pres = markers_present(frames_seen[-1]) #les mark qui sont là dans la frame
                mark=[]


                for i in mark_pres:
                    if r==0 and is_new(i, frames_seen[-1]):
                        mark.append(i) #les mark qui apparaissent avec la frame
                    elif r==-1 and is_old(i, frames_seen[-1]):
                        mark.append(i) #les mark qui disparaissent avec la frame

                #Les marqueurs qui apparaissent ou disparaissent sont ils loins des autres marqueurs présents à la frame d'apparition/de disparition ?
                for i in mark:
                    distant=True
                    for j in mark_pres:
                        if i!=j:
                            if math.dist(labdico[i].iloc[r], labdico[j].loc[frames_seen[-1]]) < dist_max:
                                distant=False
                                break                   
                    if distant:                    
                        artef.append(i)
            
    return sorted(list(dict.fromkeys(artef)))
    
#===============================================================================================
def find_artifacts3(labdico, step, dist_motion):#Supprime les marqueurs si ne bougent pas assez
    
    artef=[]
    
    for i in labdico:
        if len(labdico[i])>step:
            motion=False
            for frame in range(labdico[i].index[0], labdico[i].index[-1]-step, step):
                pos_future=labdico[i].loc[frame+step]
                if math.dist(labdico[i].loc[frame], pos_future) > dist_motion:
                    motion=True
                    break
            if not motion:
                artef.append(i)
                
    return sorted(list(dict.fromkeys(artef)))

#===============================================================================================
def find_artifacts4(labdico, step, dist_motion):#Supprime les marqueurs si ne bougent pas assez
    
    artef=[]
    
    for i in labdico:
        if len(labdico[i])>step:
            if avg_speed(i, step)<dist_motion:
                artef.append(i)
                
    return sorted(list(dict.fromkeys(artef)))

#===============================================================================================
def find_artifacts5(labdico, n_marqueurs):
    
    end_dict={} #une clé de end_dict est le nom d'une trajectoire, sa valeur est l'entier correspondant à sa dernière frame
    for i in labdico:
        end_dict[i]=labdico[i].index[-1]
    end_dict = dict(sorted(end_dict.items(), key=lambda item: item[1]))# les trajectoires qui se terminent en premier sont les premieres du dictionnaire 

    proba_artef={}
    end_fr=0
    for i in labdico:
        frame_start=labdico[i].index[0]# on regarde à l'apparition de chaque trajectoire si on a plus de 15 marqueurs
        if n_markers_present(frame_start)>n_marqueurs and frame_start>end_fr: #et après end_fr pour pas refaire les memes choses
            for j in end_dict:# on cherche j: le marqueur qui a sa disparition fait qu'il n'y a plus plus de 15 marqueurs
                if end_dict[j]>frame_start and n_markers_present(end_dict[j])<=15:
                    for k in markers_present(frame_start): #on ajoute chaque marqueurs présents à proba_artef
                        if not k in proba_artef:
                            proba_artef[k]=0

                        if len(labdico[k])>200:
                            step=100
                        else:
                            step=10

                        proba_artef[k]+=avg_speed(k, step)

                    end_fr=end_dict[j]
                    break;#quitte la boucle de end_dict car on vient de trouver le end


    if proba_artef: #si le dico n'est pas vide
        return list({k: v for k, v in sorted(proba_artef.items(), key=lambda item: item[1])}.keys())[0]
    else:
        return []

In [ ]:
def delete_artifacts(lab, labdico):
    
    artifacts = list(dict.fromkeys(find_artifacts3(labdico, 75, 3.5)+find_artifacts4(labdico, 80, 0.01625)))
    lab=[x for x in lab if x not in set(artifacts)]
    for i in artifacts:
        del labdico[i]

    artif = artifacts
    artifacts = []
    
    while True:
        artifacts=list(dict.fromkeys(find_artifacts1(labdico, 90, 670)+find_artifacts2(labdico, 625)))
        artif += artifacts
        if artifacts == []:
            break;
        lab=[x for x in lab if x not in set(artifacts)]
        for i in artifacts:
            del labdico[i]

    while True:
        artifacts=find_artifacts5(labdico, 15)
        if artifacts == []:
            break;
        else:
            lab=[x for x in lab if x not in set(artifacts)]
            del labdico[artifacts]#artifact a une taille de 1, donc pas besoin de faire de boucle
            artif += [artifacts]

    return sorted(artif)

In [ ]:
score=0
for h in ['03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
    file='Measurement'+h+'.c3d'
    n_frames, lab, labdico = getData('Measurements_unlab\\'+file)   
    file=file[:-4]        
    
    artif=delete_artifacts(lab, labdico)

    print('Measurement'+h+':', str(len(artif))+'/'+str(len(artefacts[file]))+' ', len(artif)/(len(artefacts[file])))
    
    score+=len(artif)
    
    cata = list(set(artif) - set(artefacts[file]))
    if cata != []:
        print('On a supprimé des trajectoires qui ne sont pas des artefacts !!!!!', len(cata), 'en rab')
        break;
        
print(str(score)+'/109 '+str(score/109))